In [ ]:
pip install matplotlib
pip install seaborn pandas

In [ ]:
import sqlite3
import bottle
from bottle import Bottle, run, request, redirect, template
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Initialize the Bottle app
app = Bottle()
 
# SQLite database path
db_path = 'C:/Users/asus/Desktop/Data Science/Phython and SQL/Final Project/spare_part_shop.db'

# create route for login page and def function to return the template
@app.route('/')
def login():
    return template('login', error=None)

# login form submission
@app.route('/login', method='POST')  # do_login fuction associated with the '/login' endpoint and is set to handle only HTTP POST requests.
def do_login():
    username = request.forms.get('username')
    password = request.forms.get('password')

    if validate_login(username, password):
        redirect('/main')
    else:
        return template('login', error='Login failed. Please try again.')


# Function to validate user credentials
def validate_login(username, password):
    conn = sqlite3.connect(db_path)  # ------>  eastablishing connection with the databse
    cursor = conn.cursor()   #  ------>  creates a cursor object, which is used to execute SQL queries on the connected database.
    cursor.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))  #  ------>  checking if there is a row where the 'username' and 'password' match the provided values
    user = cursor.fetchone()  #  ------>  fetches the first row which fulfill the requirement and none if it is not exist
    conn.close()
    return user is not None   #  ------>  True if a user with the provided credentials is found, indicating a successful login.



# Create the main page route and define function to return the template
@app.route('/main')
def main_page():
    return template('main')


# Function to get the list of products from the database 
def get_product_list():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Fetch the column names
    cursor.execute("PRAGMA table_info(Products)")
    columns = [column[1] for column in cursor.fetchall()]

    cursor.execute("""SELECT
        P.Product_ID,
        P.Part_Name,
        M.Manufacturer_Name,
        C.Category,
        P.Quantity,
        P.Price_Per_Unit
    FROM
        Products AS P
    JOIN
        Manufacturer AS M ON P.Manufacturer_ID = M.Manufacturer_ID
    JOIN
        Category AS C ON P.Category_ID = C.Category_ID""")  

    #cursor.execute("SELECT e.Part_Name,d.Manufacturer_Name,c.Category,e.Quantity, e.Price_Per_Unit FROM Products e inner join Manufacturer d on e.Manufacturer_ID=d.Manufacturer_ID \
     #     inner join Category c on e.Category_ID=c.Category_ID")
    products = cursor.fetchall()

    # Create a list of dictionaries with proper keys
    product_list = [dict(zip(columns, product)) for product in products]
    conn.close()
    return product_list


# Create route and function for the Product list page
@app.route('/fancy')
def fancy_page():
    product_list = get_product_list()        # Get the list of products from get_product_list function
    return template('fancy', Products=product_list)         # Pass the product list to the template



# Route to add a product page and function to get datas from
@app.route('/add_product', method='POST')
def add_product_route():
    part_name = request.forms.get('partName')
    manufacturer_id = request.forms.get('manufacturerID')
    category_id = request.forms.get('categoryID')
    quantity = request.forms.get('quantity')
    price_per_unit = request.forms.get('pricePerUnit')
    
    add_product(part_name, manufacturer_id, category_id, quantity, price_per_unit)       # Add the product to the database

    redirect('/fancy')       # Redirect to the Product list page

# Function to add a product
def add_product(part_name, manufacturer_id, category_id, quantity, price_per_unit):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO Products (Part_Name, Manufacturer_ID, Category_ID, Quantity, Price_Per_Unit) VALUES (?, ?, ?, ?, ?)",
                   (part_name, manufacturer_id, category_id, quantity, price_per_unit))
    conn.commit()
    conn.close()


# Function to retrieve employees from the database
def retrieve_employees():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Employees")
    employees = cursor.fetchall()
    conn.close()
    return employees

# Route to display the Employees page
@app.route('/employees')
def employees_page():
    employees = retrieve_employees()   # Deliveing the retrieved data to the template
    return template('employees_page', employees=employees)


# Function to retrieve loyal customers from the database
def retrieve_loyal_customers():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Customers")
    loyal_customers = cursor.fetchall()
    conn.close()
    return loyal_customers

# Route to display the Loyal Customers page and function that return the template
@app.route('/loyal_customers')
def loyal_customers_page():
    loyal_customers = retrieve_loyal_customers()   # delivering the data to the template 
    return template('loyal_customers', loyal_customers=loyal_customers)



# Route for the statistics page
@app.route('/statistics')
def statistics_page():
    try:
        # Connect to the database for the first query
        conn = sqlite3.connect(db_path)

        # Create a cursor object for the first query
        cursor = conn.cursor()

        # Fetch the total sales for each employee
        cursor.execute("""
            SELECT Employees.Name, SUM([Transaction].Qty * Products.Price_Per_Unit) AS TotalSales
            FROM [Transaction]
            JOIN Employees ON [Transaction].Employee_ID = Employees.Employee_ID
            JOIN Products ON [Transaction].Product_ID = Products.Product_ID
            GROUP BY Employees.Name
        """)

        # Fetch all the results for the first query
        sales_data = cursor.fetchall()

        # Close the database connection for the first query
        conn.close()

        # Create a more fancy histogram using Seaborn and Matplotlib for the first query
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Employee Name', y='Total Sales', data=pd.DataFrame(sales_data, columns=['Employee Name', 'Total Sales']), palette='viridis')
        plt.xlabel('Employee Name', fontsize=15)
        plt.ylabel('Total Sales', fontsize=15)
        plt.title('Total Sales by Employee', fontsize=16)
        plt.xticks(rotation=45, ha='right', fontsize=12)  # Increase font size for x-axis labels
        plt.yticks(fontsize=12)  # Increase font size for y-axis labels

        # Save the plot to a file for the first query
        plt.savefig('static/fancy_histogram.png')

        # Connect to the database again for the second query
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Fetch the count of products in each category for the second query
        cursor.execute("""
            SELECT Category.Category, COUNT(*) AS ProductCount
            FROM Products
            JOIN Category ON Products.Category_ID = Category.Category_ID
            GROUP BY Category.Category
        """)

        # Fetch all the results for the second query
        results = cursor.fetchall()

        # Close the database connection for the second query
        conn.close()

        # Generate histogram using matplotlib for the second query
        plt.figure(figsize=(12, 8))
        categories = [row[0] for row in results]
        product_counts = [row[1] for row in results]
        colors = sns.color_palette('viridis', len(categories))  # Use the same color palette as the first histogram
        plt.bar(categories, product_counts, color=colors)
        plt.xlabel('Category', fontsize=15)
        plt.ylabel('Product Count', fontsize=15)
        plt.title('Product Distribution Across Categories', fontsize=16)
        plt.xticks(fontsize=12)  # Increase font size for x-axis labels
        plt.yticks(fontsize=12)  # Increase font size for y-axis labels

        # Save the plot to a file for the second query
        plt.savefig('static/histogram.png')

        # Pass the image file paths to the template
        fancy_image_path = 'static/fancy_histogram.png'
        simple_image_path = 'static/histogram.png'

        return template('statistics', fancy_image_path=fancy_image_path, simple_image_path=simple_image_path)

    except sqlite3.Error as e:
        print("SQLite error:", e)
        return template('error', message='An error occurred while fetching statistics.')

from bottle import route, static_file

@app.route('/static/:path#.+#', name='static')
def static(path):
    return static_file(path, root='./static')
    
if __name__ == '__main__':
    #create_table()
    run(app, host='localhost', port=8080, debug=True)


Bottle v0.12.25 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [02/Mar/2024 23:34:01] "GET / HTTP/1.1" 200 2301
Traceback (most recent call last):
  File "C:\Users\asus\anaconda3\Lib\site-packages\bottle.py", line 876, in _handle
    return route.call(**args)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\asus\anaconda3\Lib\site-packages\bottle.py", line 1759, in wrapper
    rv = callback(*a, **ka)
         ^^^^^^^^^^^^^^^^^^
  File "C:\Users\asus\AppData\Local\Temp\ipykernel_22796\202054581.py", line 25, in do_login
    if validate_login(username, password):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\asus\AppData\Local\Temp\ipykernel_22796\202054581.py", line 35, in validate_login
    cursor.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))  #  ------>  checking if there is a row where the 'username' and 'password' match the provided values
    ^^^^^^^^^^^^^^^^^^^^^